In [1]:
import json
import pandas as pd
from pandas import json_normalize

In [2]:
with open('../Data/authors.json', 'r', encoding='utf-8') as fin:
    authors = json.loads(fin.read())

In [3]:
with open('../Data/books.json', 'r', encoding='utf-8') as fin:
    books = json.loads(fin.read())

In [4]:
with open('../Data/responses.json', 'r', encoding='utf-8') as fin:
    responses = json.loads(fin.read())

In [5]:
authors_df = pd.DataFrame(authors, columns=['name', 'link'])

In [6]:
books_df = pd.DataFrame(books, columns=['name', 'author', 'category', 
                                'link', 'avg_rating', 'responses_amount'])

In [7]:
resp_df = json_normalize(responses, record_path=['responses'], 
                          meta=['name', 'author', 'annotation'])
resp_df = resp_df.reindex(columns=['name', 'author', 'annotation', 'text', 'mark', 'hasSpoiler'])

In [8]:
resp_df.loc[resp_df.hasSpoiler == 1].tail()

,name,author,annotation,text,mark,hasSpoiler
84371,Мёртвая линия. Оно начинается...,Джонатан и Саймон Уокер,"Шли первые сутки из девяти, названных в Пророч...",Оно началось... Что именно? Русская пародия на...,3,1
84373,Мёртвая линия. Оно приближается…,Джонатан и Саймон Уокер,"Когда времени на раздумья уже не осталось, при...","Вторая книга трилогии. В целом, достойное прод...",2,1
84375,Мёртвая линия. Оно приближается…,Джонатан и Саймон Уокер,"Когда времени на раздумья уже не осталось, при...","Мне книга очень понравилась!!! Легко читается,...",1,1
84376,Мёртвая линия. Оно вершится,Джонатан и Саймон Уокер,"Последние трое суток, отведённые Пророчеством ...",Хорошее завершение истории. Особенно порадовал...,4,1
84395,"Кровавое заклятие, или Акация",Дэвид Энтони Дарем,Много веков назад империя Акация правила бесчи...,"<spoiler> Вот сейчас с удивлением осознал, ч...",1,1


# Class ratio

In [11]:
resp_df['hasSpoiler'].mean()

0.06406246297481102

# The length of spoiler/non-spoiler texts

In [24]:
from nltk.tokenize import word_tokenize

In [10]:
df = pd.read_csv('../Data/train_data.csv', lineterminator='\n')

In [26]:
df['text']

0        А жаль — могла получится вполне интересная и н...
1        Должен сказать, что прочтение «Черновика» меня...
2        «А внутре у ней неонка» (с.)   Когда-то, лет э...
3        Эта книга никак не дотягивает до хорошей фанта...
4        Вещь не вторичная, а даже третичная. Если твор...
                               ...                        
84397    Переосмысление сказок – это всегда интересно. ...
84398    Занятный альтернативный взгляд на классическую...
84399    Самый характерный пример убитого нашими книгои...
84400    Мое самое любимое произведение. Читал, не отры...
84401    Мой любимая трилогия! Читается легко и интерес...
Name: text, Length: 84402, dtype: object

In [35]:
spoiler_df = df.loc[df['hasSpoiler'] == 1]
non_spoiler_df = df.loc[df['hasSpoiler'] == 0]

In [36]:
def text2tokens(text):
    return [word for word in word_tokenize(text) if word.isalpha()]

In [37]:
spoiler_tokens = spoiler_df['text'].apply(text2tokens)
non_spoiler_tokens = non_spoiler_df['text'].apply(text2tokens)

spoiler_texts_length = pd.Series([len(tok) - 1 for tok in spoiler_tokens])
non_spoiler_texts_length = pd.Series([len(tok) for tok in non_spoiler_tokens])

In [38]:
spoiler_texts_length.describe()

count    5407.000000
mean      254.865730
std       231.645199
min         5.000000
25%       106.000000
50%       191.000000
75%       331.000000
max      4932.000000
dtype: float64

In [40]:
non_spoiler_texts_length.describe()

count    78995.000000
mean       155.649611
std        182.028087
min          0.000000
25%         46.000000
50%         92.000000
75%        196.000000
max       4554.000000
dtype: float64

In [41]:
non_spoiler_texts_length.median()

92.0

# Names detection

In [42]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [43]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [44]:
def get_names_amount(text):
    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    for span in doc.spans:
        span.normalize(morph_vocab)

    for span in doc.spans:
        if span.type == PER:
            span.extract_fact(names_extractor)

    lemmatized_text = ' '.join([_.lemma for _ in doc.tokens if _.rel != 'punct'])

    return {
        'lemmatized_text': lemmatized_text,
        'names_amount': len([name.fact for name in doc.spans 
                if name.type == PER and name.fact is not None])
    }

In [45]:
from tqdm import tqdm

In [ ]:
result = {'lemmatized_text': [], 'names_amount': []}

resp_df.to_csv('train_data.csv', header=False)
for text in tqdm(resp_df['text']):
    res = get_names_amount(text)
    
    result['lemmatized_text'].append(res['lemmatized_text'])
    result['names_amount'].append(res['names_amount'])

In [13]:
resp_df['lemmatized_text'] = result['lemmatized_text']
resp_df['names_amount'] = result['names_amount']

In [16]:
resp_df.to_csv('train_data1.csv', index=False)

In [21]:
df = pd.read_csv('train_data1.csv', lineterminator='\n')